In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score,f1_score
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import six
import sys
sys.modules['sklearn.externals.six'] = six

In [3]:
df = pd.read_csv("UNSW_undersampled_and_reversed.csv")
df

,Unnamed: 0,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,0,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,1,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,2,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,3,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,4,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185995,185995,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,15,42,0,0,0,30,42,0,Generic,1
185996,185996,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1
185997,185997,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1
185998,185998,0.000008,udp,dns,INT,2,0,114,0,125000.000300,...,23,35,0,0,0,35,35,0,Generic,1


In [10]:
from sklearn.utils import shuffle
df = shuffle(df,random_state=50)

In [9]:
from sklearn import preprocessing

In [11]:
label_encoder1 = preprocessing.LabelEncoder()
df['proto']= label_encoder1.fit_transform(df['proto'])
  
df['proto'].unique()
label_encoder2 = preprocessing.LabelEncoder()
df['attack_cat']= label_encoder2.fit_transform(df['attack_cat'])
  
df['attack_cat'].unique()
label_encoder3 = preprocessing.LabelEncoder()
df['service']= label_encoder3.fit_transform(df['service'])
  
df['service'].unique()
label_encoder4 = preprocessing.LabelEncoder()
df['state']= label_encoder4.fit_transform(df['state'])
  
df['state'].unique()

array([3, 4, 1, 6, 7, 2, 0, 5, 8, 9])

In [12]:
y=df['attack_cat']
#y1=df.iloc[:,-1]
x_k=df.drop(['label','attack_cat','Unnamed: 0'],axis=1)
#y= df['attack_cat']

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_l = le.fit_transform(y)
y_l

array([6, 6, 6, ..., 3, 6, 5], dtype=int64)

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y_l = le.fit_transform(y)
y_l

array([6, 6, 6, ..., 3, 6, 5], dtype=int64)

In [15]:
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
x_k_1= sc.fit_transform(x_k)

x = pd.DataFrame(x_k_1)

x.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,8.793018e-03,0.856061,0.0,0.333333,0.013062,0.003497,0.001504,0.000119,2.843165e-05,0.996078,...,0.017241,0.017241,0.000000,0.048387,0.0,0.0,0.0000,0.016949,0.049180,0.0
1,6.524668e-04,0.856061,0.0,0.333333,0.047896,0.020979,0.006506,0.005907,1.762542e-03,0.121569,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.084746,0.049180,0.0
2,9.706618e-03,0.856061,0.5,0.333333,0.013062,0.004662,0.002271,0.000491,2.918971e-05,0.243137,...,0.000000,0.000000,0.000000,0.064516,0.0,0.0,0.0625,0.000000,0.065574,0.0
3,5.008349e-01,0.590909,0.0,0.444444,0.027576,0.000000,0.003704,0.000000,6.322780e-07,0.996078,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0000,0.000000,0.000000,0.0
4,1.000000e-07,0.909091,0.0,0.444444,0.001451,0.000000,0.000519,0.000000,1.666667e-01,0.996078,...,0.051724,0.051724,0.066667,0.064516,0.0,0.0,0.0000,0.101695,0.065574,0.0


In [16]:
from sklearn.model_selection import train_test_split

In [17]:
from sklearn.model_selection import RandomizedSearchCV

In [18]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
parameters_RF = {'n_estimators': (40,60,65,70,75,80,85,90,95,100,120),'min_samples_split' : (2,4,8,10,15,20,30)}
                                                                                                                                     
RF_grid  = RandomizedSearchCV( RandomForestClassifier(), param_distributions = parameters_RF, cv = 5, verbose = True,scoring='accuracy')
result_RF=RF_grid.fit(x,y)
  
RF= result_RF.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [14]:
RF_estimator_result =result_RF.best_estimator_
print(RF_estimator_result)

RandomForestClassifier(min_samples_split=30, n_estimators=75)


In [19]:
from sklearn.naive_bayes import GaussianNB

In [16]:
parameters_gb = {'var_smoothing': (1e-09,1e-08,1e-07,1e-06,1e-05,1e-04,1e-03,1e-02,1e-01,1,10,5e-09,5e-08,5e-07,5e-06,5e-05,
                                5e-04,5e-03,5e-02,5e-01,5)}
gb_grid  = RandomizedSearchCV( GaussianNB(), param_distributions = parameters_gb, cv = 5, verbose = True,scoring='accuracy')
result_gb = gb_grid.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [17]:
gb_estimator_result =result_gb.best_estimator_
print(gb_estimator_result)

GaussianNB(var_smoothing=0.1)


In [20]:
from sklearn.linear_model import LogisticRegression

In [19]:
parameters_lr = {'solver':('liblinear','newton-cg'),'random_state':(0,1),'C': (0.1,1,10),'tol': (0.01,0.001,0.0001,.1)}
                                                                                                                                     
lr_grid  = RandomizedSearchCV( LogisticRegression(), param_distributions = parameters_lr, cv = 5, verbose = True,scoring='accuracy')
result_lr=lr_grid.fit(x,y)
    
LG_estimator_result = result_lr.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [20]:
print(LG_estimator_result)

LogisticRegression(C=10, random_state=0, solver='newton-cg')


In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
parameters_knn = {'n_neighbors': (1,2,3,4,5),'leaf_size':(15,30,50,100),'p':(1,2,3)}
knn_grid  = RandomizedSearchCV( KNeighborsClassifier(), param_distributions = parameters_knn, cv = 5, verbose = True,scoring='accuracy')
result_knn=knn_grid.fit(x,y)
knn_estimator_result = result_knn.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [23]:
print(knn_estimator_result)

KNeighborsClassifier(leaf_size=15, p=1)


In [22]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
parameters_dt = {'min_samples_split' : (2,4,6,10), 'max_depth':(3,4,5,6,7,8,9,10,11,12,16,25,50,100,200)}                                                                                                                            
DT_grid  = RandomizedSearchCV( DecisionTreeClassifier(), param_distributions = parameters_dt, cv = 5, verbose = True,scoring='accuracy')
result_dt=DT_grid.fit(x,y)
    
dt_estimator_result = result_dt.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [32]:
print(dt_estimator_result)

DecisionTreeClassifier(max_depth=12, min_samples_split=4)


In [24]:
from xgboost import XGBClassifier

ModuleNotFoundError: No module named 'xgboost'

In [25]:
import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [26]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("ggplot")

import xgboost as xgb

ModuleNotFoundError: No module named 'xgboost'

In [20]:
from sklearn.ensemble import GradientBoostingClassifier

In [43]:
parameters_grb = {'n_estimators': (50,100,120,140,150),'min_samples_split' : (2,4),'subsample' : (0.1,0.5,1)}
                                                                                                                                     
grb_grid  = RandomizedSearchCV( GradientBoostingClassifier(learning_rate=10000), param_distributions = parameters_grb, cv = 5, verbose = 50,scoring='accuracy')
result=grb_grid.fit(x,y)
    
grb_estimator_result = result.best_estimator_

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 1/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.116 total time= 9.9min
[CV 2/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 2/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.828 total time= 9.9min
[CV 3/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 3/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.095 total time=10.2min
[CV 4/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 4/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.018 total time= 9.9min
[CV 5/5; 1/10] START min_samples_split=2, n_estimators=140, subsample=1.........
[CV 5/5; 1/10] END min_samples_split=2, n_estimators=140, subsample=1;, score=0.038 total time=10.2min
[CV 1/5; 2/10] STAR

In [44]:
print(grb_estimator_result)

GradientBoostingClassifier(learning_rate=10000, n_estimators=140, subsample=0.1)


In [21]:
from sklearn.ensemble import AdaBoostClassifier

ModuleNotFoundError: No module named 'kerastuner'

In [48]:
pip install keras-tuner 

Note: you may need to restart the kernel to use updated packages.


In [22]:
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt

In [23]:
numpy_array_X_test_a = x_2.to_numpy()

NameError: name 'x_2' is not defined

In [24]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
label=le.fit_transform(y_2)

NameError: name 'y_2' is not defined